In [2]:
import pandas as pd, numpy as np, re, os

FIVE_FILE = "five_gene_matrix.csv" 
META_CSV  = "sample_metadata.csv"
CLIN_XLS  = "GSE24080_MM_UAMS565_ClinInfo_27Jun2008_LS_clean.xls"
OUT_CSV   = "final_columns.csv"

five = pd.read_csv(FIVE_FILE, dtype=str)
if "Unnamed: 0" in five.columns:
    five = five.rename(columns={"Unnamed: 0": "Gene"}).set_index("Gene")
else:
    five = five.set_index(five.columns[0])
five = five.apply(pd.to_numeric, errors="coerce")
five_T = five.T.reset_index().rename(columns={"index": "GSM_ID"}) 

meta = pd.read_csv(META_CSV, dtype=str)

m1 = meta.merge(five_T, on="GSM_ID", how="left")

def extract_celfilename(path: str) -> str:
    return path.split("_")[-1][:-3]

m1["CELfilename"] = m1["supplementary_file"].astype(str).apply(extract_celfilename)

clin = pd.read_excel(CLIN_XLS, dtype=str)
clin.columns = [c.strip() for c in clin.columns]

if "CELfilename" not in clin.columns:
    raise ValueError("Clinical XLS does not contain 'CELfilename' column.")
if "PATID" not in clin.columns:
    raise ValueError("Clinical XLS does not contain 'PATID' column.")

m2 = m1.merge(clin, on="CELfilename", how="left")

def to_event(v):
    return int(v=="1")

def to_num(v):
    if v == "<0.5": return 0.5  #Special case in B2m in the file
    return float(v)

OS_event = m2["OS-censor"].apply(to_event)
OS_time  = m2["OS (months)"].apply(to_num)
EFS_event = m2["EFS-censor"].apply(to_event)
EFS_time  = m2["EFS(months)"].apply(to_num)

def iss_stage(b2m, alb):
    if pd.isna(b2m) or pd.isna(alb): return np.nan
    if b2m < 3.5 and alb >= 3.5: return "I"
    if (b2m < 3.5 and alb < 3.5) or (3.5 <= b2m <= 5.5): return "II"
    if b2m > 5.5: return "III"
    return np.nan

B2m = m2["B2M"].apply(to_num)
Albumin = m2["ALB"].apply(to_num)
ISS = [iss_stage(b, a) for b, a in zip(B2m, Albumin)]

final = pd.DataFrame({
    "Patient_ID":   m2["PATID"],
    "EPAS1":        m2["EPAS1"],
    "ERC2":         m2["ERC2"],
    "PRC1":         m2["PRC1"],
    "CSGALNACT1":   m2["CSGALNACT1"],
    "CCND1":        m2["CCND1"],
    "OS_event":     OS_event,
    "OS_time":      OS_time,
    "EFS_event":     EFS_event,
    "EFS_time":      EFS_time,
    # "B2m":          B2m,
    # "Albumin":      Albumin,
    "ISS Staging":  ISS,  
})

final.to_csv(OUT_CSV, index=False)
print(f"Done. Wrote {OUT_CSV} with {len(final)} rows.")
display(final.head())


Done. Wrote final_columns.csv with 559 rows.


,Patient_ID,EPAS1,ERC2,PRC1,CSGALNACT1,CCND1,OS_event,OS_time,EFS_event,EFS_time,ISS Staging
0,8241,6.6409,6.8256,9.7622,12.1424,13.1576,0,93.0,0,93.0,I
1,9843,10.3075,9.3064,8.5312,10.7866,11.8152,0,96.0,0,96.0,I
2,9888,7.6385,6.6543,9.0621,13.2617,12.7715,0,87.0,0,87.0,II
3,9835,8.0128,7.4137,8.0636,11.5271,3.9054,0,98.0,1,75.0,I
4,9997,6.7282,11.6046,8.5189,10.9782,6.1372,1,43.0,0,42.0,II
